In [ ]:
import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm
import dask.bag as db
import json

In [ ]:
from dask.distributed import Client
client = Client()

In [ ]:
client

In [ ]:
bucketname = 'borza-hotelcom-data'

In [ ]:
def parse_review_block(soup,hid):
    revcont = soup.find('div',{'class':'review-container'})
    result = {}
    if type(revcont) == type(None):
        return {}
    for fun, name in [[lambda x: x.find('div',{'class':'heading'}
                        ).find('span',{'class':'badge'}).text,'review_badge'],
                      [lambda x: x.find('div',{'class':'heading'}
                        ).find('span',{'class':'rating'}).text,'review_hotels-rating'],
                      [lambda x: x.find('span',{'class':'ta-logo'}).text,'review_ta-rating'],
                      [lambda x: x.find('span',{'class':'ta-total-reviews'}).text,'review_ta-revs'],
                      [lambda x: x.find('a',{'class':'total-reviews'}).text,'review_hotels-revs']]:
        try:
            result[name] = fun(revcont)
        except:
            #print('FUCKED ----------- %s --- %s' % (name,hid))
            pass
    
    for revdiv in revcont.find_all('div',{'class':'trust-you-reviews'}):
        for rev in revdiv.find_all('div',{'class':'review'}):
            revname = 'review_' + rev.find('div',{'class':'category-name'}).text
            try:
                result[revname + '-score'] = rev.find('div',{'class':'score'})['style']
            except:
                #print(revname,'score',hid)
                pass
            try:
                result[revname + '-text'] = rev.find('div',{'class':'text'}).text
            except:
                #print(revname,'text',hid)
                pass
    return result

In [ ]:
def process_hotel_attributes(hotelid):
    hotelid = str(hotelid)
    response = requests.get('https://www.hotels.com/ho' + hotelid,
                                     params={'locale':'en_IE'})
    result = {'hotelid':hotelid,
             'date-downloaded':datetime.date.today().isoformat(),
             'response_code':response.status_code}
    soup = BeautifulSoup(response.content,'html5lib')
    for vcard in soup.find_all('div',{'class':'property-description'}):
        for fun, name in [[lambda x: x.find('h1').text,'name'],
                         [lambda x: x.find('span',{'class':'hotel-coordinates'}
                                          ).find_all('meta')[0]['content'],'lat'],
                         [lambda x: x.find('span',{'class':'hotel-coordinates'}
                                          ).find_all('meta')[1]['content'],'lon'],
                         [lambda x: x.find('span',{'class','star-rating-text'}).text,'star'],
                         [lambda x: x.find('span',{'class','street-address'}).text,'street'],
                         [lambda x: x.find('span',{'class','locality'}).text,'locality'],
                         [lambda x: x.find('span',{'class','region'}).text,'region'],
                         [lambda x: x.find('span',{'class','postal-addr'}).text,'addr'],
                         [lambda x: x.find('span',{'class','postal-code'}).text,'zip'],
                         [lambda x: x.find('span',{'class','country-name'}).text,'country'],
                         [lambda x: x.find('div',{'class','tagline'}).text,'tagline']]:
            try:
                result[name] = fun(vcard)
            except:
                #print('FUCKED ----------- %s --- %s' % (name,hotelid))
                pass
    try:
        result['dest-id'] = soup.find('span',{'class':'back-link'}).find('a')['href'].split(
            'destination-id=')[-1].split('&')[0]
    except:
        #print('FUCKED ----------- %s --- %s' % ('dest-id',hotelid))
        pass
    
    for findict,name in [[{'data-overview-section-type':"HOTEL_FEATURE"},'base'],
                        [{'data-overview-section-type':"FAMILY_FRIENDLY_SECTION"},'family'],
                        [{'class':'key-facts-container'},'key'],
                        [{'class':'travelling-container'},'travel'],
                        [{'class':'transport-container'},'transport'],
                        [{'class':'badges-and-services-container'},'badges']]:
        baselist = soup.find('div',findict)
        if name == 'key':
            try:
                baselist.find('div',{'class':'expandable-wrapper'}).decompose()
            except:
                pass
        try:
            lis = baselist.find_all('li')
        except:
            #print('FUCKED ----------- %s --- %s' % (name,hotelid))
            lis = []
            result[name + '_missing'] = 1
        for li in lis:
            result[name + '_' + li.text] = 1


    for findict,name in [[{'class':'fact-sheets in-the-property-module'},'inhotel'],
                         [{'class':'fact-sheets in-the-room-module'},'inroom']]:    
        inhotel = soup.find('div',findict)
        try:
            lis = inhotel.find_all('div',{'class':'fact-sheet-table-row'})
        except:
            #print('FUCKED ----------- %s --- %s' % (name,hotelid))
            result[name + '_missing'] = 1
            lis = []
        for li in lis:
            head = li.find('div',{'class':'fact-sheet-table-header'})
            try:
                head = head.text
                cells = li.find_all('li')
            except:
                #print('FUCKED ----------- %s --- %s' % (name,hotelid))
                cells = []
            for item in cells:
                result[name + '_' + item.text] = 1
    result.update(parse_review_block(soup,hotelid))
    return result

In [ ]:
ATTS_SAMPLE = 0.0002

In [ ]:
full_sitemap = pd.read_csv('s3://%s/full_sitemap.csv' % bucketname,low_memory=False)

In [ ]:
hotel_sample = db.from_sequence(full_sitemap\
                                .loc[full_sitemap['term'] == 'HOTEL','id']\
                                .sample(frac=ATTS_SAMPLE).values)

In [ ]:
hotel_sample.map(process_hotel_attributes).map(json.dumps)\
    .to_textfiles("s3://%s/hotel-atts-*.json" % bucketname)